# Ex 1: Manipulating Spectra Part 1 - `Spectrum`

# Notebook Initialization

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%autoreload
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [10,10]

import numpy as np
from scipy.integrate import quad

import darkhistory.physics as physics
import darkhistory.spec.spectools as spectools
from darkhistory.spec.spectrum import Spectrum

## Introduction

The manipulation of spectral data in this code is done through several specialized classes, all of them equipped with internal methods that make data manipulation easy. Functions useful for creating and manipulating these classes are provided in `spec.spectools`. 

## `Spectrum` Class - Introduction

The most basic class that we use is the `Spectrum` class. This contains an abscissa, the spectrum values at each abscissa, as well as the redshift. 

To create our first `Spectrum`, we will use the function `spec.spectools.discretize`. This takes a function which returns the $dN/dE$ of a continuous spectrum, and produces a discretized version of it, stored in a `Spectrum` class. 

Our first example will be a `Spectrum` containing a CMB spectrum. `physics.CMB_spec` returns $dN/dE$ at a particular energy and temperature of the blackbody spectrum. This function can be passed to `spec.spectools.discretize` to create a `Spectrum` object.

In [3]:
T = 1.5 # The temperature in eV of the CMB.

def CMB_dNdE_at_T(eng):
    return physics.CMB_spec(eng, T)

eng = 10**((np.arange(120)-90)*(1/10))
discrete_CMB = spectools.discretize(CMB_dNdE_at_T, eng)

AttributeError: 'function' object has no attribute 'size'

`discrete_CMB` is a `Spectrum` object. This object has several attributes, but the three most important ones are the energy abscissa `self.eng`, the $dN/dE$ value at each point `self.dNdE`, as well as the redshift (always $1+z$ in this project) `self.rs`.

`self.eng` should be viewed as the center of a bin. The bin boundary between two successive abscissa values $E_1$ and $E_2$ is $\frac{1}{2} ( \log E_1 + \log E_2)$. The first and last abscissa values are taken to lie at the center of their respective bins.  

We can now plot the spectrum contained in `discrete_CMB` as a check. 

In [ ]:
ax = plt.subplot(1,1,1)
plt.rc('text', usetex=True)

discrete_spec, = plt.plot(
    discrete_CMB.eng, discrete_CMB.dNdE, label='Discrete Spectrum'
)
analytic_spec, = plt.plot(
    eng, physics.CMB_spec(eng, T), 'go', marker='o', markersize='10', 
    markevery=3, markerfacecolor='w', label='Analytic Spectrum'
)

plt.legend(handles=[discrete_spec, analytic_spec], 
           fontsize=24)

ax.set_xscale('log')
ax.set_yscale('log')

plt.tick_params(axis='both', which='major', labelsize=20)

plt.title('CMB Spectrum', fontsize=28)
plt.xlabel('Photon Energy [eV]',fontsize=24)
plt.ylabel('dN/(dE dV) [eV$^{-1}$ cm$^{-3}$]',fontsize=24)
plt.axis([1e-8, 1e4, 1e-10, 1e20])

We should also set the `rs` attribute to the correct redshift. Here, we use `physics.TCMB`, which takes in a redshift as an argument, to calculate what the redshift $1+z$ is at the temperature `T` that we selected earlier. 

In [ ]:
rs = T/physics.TCMB(1)

discrete_CMB.rs = rs

## `Spectrum` Class - Addition and Multiplication

You can add two `Spectrum` objects together, or add a scalar to a `Spectrum`. Similarly, you can multiply two `Spectrum` objects, or multiply by a scalar. These operations can be performed as if `Spectrum` were an array. 

## `Spectrum` Class - Number and Energy

Let's initialize a new `Spectrum` that is essentially a copy of `discrete_CMB`. 

In [ ]:
test_CMB = Spectrum(discrete_CMB.eng, discrete_CMB.dNdE, discrete_CMB.rs)

To find the total number of particles stored in this spectrum, we can use the method `self.totN()`, with no arguments. Particle number in a bin is always given by 

$$ N = \frac{dN}{dE} \times E \, d\log E$$

where $d \log E$ is the log bin width of the bin (this can be computed with `spec.spectools.get_log_bin_width`).

This agrees with the actual numerical quadrature over the CMB spectrum given by `physics.CMB_spec` from `eng[0]` to `eng[-1]`, since `spec.spectools.discretize` ensures that the total number of particles *and* total energy in each bin are exactly the value that would be obtained by quadrature of the CMB spectrum. 

In [ ]:
print('Number of particles stored in test_CMB: ', test_CMB.totN())
print('Number of particles by quadrature: ', quad(CMB_dNdE_at_T, eng[0], eng[-1])[0])

`self.totN` can also be used to find the total number of particles in various bins. This is done by specify the bin boundaries: the code below returns the total number of particles in the first two bins. The bin boundaries do not need to be integers. 

In [ ]:
print('# of particles in first and second bin: ',
      test_CMB.totN(bound_type='bin', bound_arr=np.array([0,1,2]))
     )

`self.totN` can also return the total number of particles between different energy boundaries: 

In [ ]:
print('# of particles between 0.2 and 0.45 eV, 0.45 eV and 0.6 eV: ', 
      test_CMB.totN(bound_type='eng', bound_arr=np.array([0.2, 0.45, 0.6]))
     )

The function `self.toteng()` serves a similar purpose. Energy in a bin is always evaluated as
$$ E = \frac{dN}{dE} \times \, E^2 d \log E $$

In [ ]:
def CMB_EdNdE_at_T(eng):
    return eng*physics.CMB_spec(eng, T)

print('Total energy stored in test_CMB: ', test_CMB.toteng())
print('Total energy by quadrature: ', quad(CMB_EdNdE_at_T, eng[0], eng[-1])[0])

## `Spectrum` Class - Redshifting and Underflow

Photon redshifting is built in to the `Spectrum` class.

In [ ]:
test_CMB.redshift(rs/2)

The resulting redshifted spectrum conserves total number, with the total energy reduced by half. Here is a plot of the new spectrum, and a comparison of the total number and total density.

In [ ]:
ax = plt.subplot(1,1,1)
plt.rc('text', usetex=True)

orig_spec, = plt.plot(discrete_CMB.eng, discrete_CMB.dNdE, label='Original Spectrum')
redshifted_spec, = plt.plot(test_CMB.eng, test_CMB.dNdE/2**3, label='Redshifted Spectrum')

plt.legend(handles=[orig_spec, redshifted_spec], fontsize=24)

ax.set_xscale('log')
ax.set_yscale('log')

plt.tick_params(axis='both', which='major', labelsize=20)

plt.title('CMB Spectrum', fontsize=28)
plt.xlabel('Photon Energy [eV]',fontsize=24)
plt.ylabel('dN/(dE dV) [eV$^{-1}$ cm$^{-3}$]',fontsize=24)
plt.axis([1e-8, 1e4, 1e-10, 1e20])

print('Original total number of particles: ', discrete_CMB.totN())
print('Redshifted total number of particles: ', test_CMB.totN())
print('Ratio: ', discrete_CMB.totN()/test_CMB.totN())
print('**********************************************************')
print('Original total energy: ', discrete_CMB.toteng())
print('Redshifted total energy: ', test_CMB.toteng())
print('Ratio: ', discrete_CMB.toteng()/test_CMB.toteng())


Note that the energy abscissa doesn't change after redshifting. Number and energy are both conserved by assigning particles that redshift below the energy abscissa into an underflow. Underflow number and energy are included in `self.totN()` and `self.toteng()`. 

In [ ]:
print('Number of particles in underflow: ',test_CMB.underflow['N'])
print('Energy in underflow: ', test_CMB.underflow['eng'])

## `Spectrum` - Rebinning and Shifting

The energy abscissa of a `Spectrum` object can be adjusted. 

If every photon in a bin, with energies $E_1, \cdots, E_n$, loses some amount of energy, and now has new energies $E_1', \cdots, E_n'$, the abscissa can be shifted down to the new abscissa, while conserving number, using the function `self.shift_eng`. 

A closely related function is `self.rebin`. This re-distributes the spectrum into a new abscissa, while conserving number *and* energy. The new abscissa must have a maximum energy higher than the old abscissa, so that there is no overflow. 

As an example, we will perform redshifting explicitly using these two functions from $1+z$ to $(1+z)/2$. Photons at an energy $E_n$ are shifted to energy $E_n/2$ using `self.shift_eng`. Then, we rebin the resulting spectrum back into the original abscissa to get the redshifted spectrum using `self.rebin`. The result is identical to calling the `self.redshift` function.

In [ ]:
test_CMB_2 = Spectrum(discrete_CMB.eng, discrete_CMB.dNdE, discrete_CMB.rs)
test_CMB_2.shift_eng(discrete_CMB.eng/2)
test_CMB_2.rebin(discrete_CMB.eng)

ax = plt.subplot(1,1,1)
plt.rc('text', usetex=True)

orig_spec, = plt.plot(discrete_CMB.eng, 
                      discrete_CMB.dNdE, 
                      label='Original Spectrum')
rebinned_spec, = plt.plot(test_CMB_2.eng, 
                          test_CMB_2.dNdE/2**3, 
                          label='Shifted and Rebinned Spectrum')
redshifted_spec, = plt.plot(test_CMB.eng, 
                            test_CMB.dNdE/2**3,'go',
                            marker='o', markersize='10', 
                            markevery=3, markerfacecolor='w', 
                            label='Redshifted Spectrum')


plt.legend(handles=[orig_spec, rebinned_spec, redshifted_spec], 
           fontsize=24)

ax.set_xscale('log')
ax.set_yscale('log')

plt.tick_params(axis='both', which='major', labelsize=20)

plt.title('CMB Spectrum', fontsize=28)
plt.xlabel('Photon Energy [eV]',fontsize=24)
plt.ylabel('dN/(dE dV) [eV$^{-1}$ cm$^{-3}$]',fontsize=24)
plt.axis([1e-8, 1e4, 1e-10, 1e22])

print('Original total number of particles: ', discrete_CMB.totN())
print('Redshifted total number of particles: ', test_CMB_2.totN())
print('Ratio: ', discrete_CMB.totN()/test_CMB_2.totN())
print('**********************************************************')
print('Original total energy: ', discrete_CMB.toteng())
print('Redshifted total energy: ', test_CMB_2.toteng())
print('Ratio: ', discrete_CMB.toteng()/test_CMB_2.toteng())
